In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
tf.random.set_seed(42)

In [4]:
X, y = fetch_california_housing(return_X_y=True)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
normalize_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:]) # alt for flatten; behaves like a standard scaler.

In [8]:
model = tf.keras.Sequential([
    normalize_layer,
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1),
])

2024-01-27 22:42:07.750442: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-01-27 22:42:07.750475: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-01-27 22:42:07.750485: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-01-27 22:42:07.752097: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-27 22:42:07.755437: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
model.compile(loss="mse", optimizer="adam", metrics=["RootMeanSquaredError"])

In [10]:
normalize_layer.adapt(X_train) # adapt to mean and sd of training data; to replicate standard scaler.

2024-01-27 22:43:58.526635: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [11]:
hist = model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30
516/516 [==============================] - 7s 10ms/step - loss: 1.2082 - root_mean_squared_error: 1.0992 - val_loss: 0.5809 - val_root_mean_squared_error: 0.7622
Epoch 2/30
516/516 [==============================] - 6s 12ms/step - loss: 1.5802 - root_mean_squared_error: 1.2571 - val_loss: 0.6991 - val_root_mean_squared_error: 0.8361
Epoch 3/30
516/516 [==============================] - 5s 10ms/step - loss: 3.6809 - root_mean_squared_error: 1.9186 - val_loss: 0.5730 - val_root_mean_squared_error: 0.7570
Epoch 4/30
516/516 [==============================] - 6s 12ms/step - loss: 4.5610 - root_mean_squared_error: 2.1357 - val_loss: 0.5669 - val_root_mean_squared_error: 0.7529
Epoch 5/30
516/516 [==============================] - 7s 13ms/step - loss: 1.3008 - root_mean_squared_error: 1.1405 - val_loss: 0.6258 - val_root_mean_squared_error: 0.7911
Epoch 6/30
516/516 [==============================] - 5s 10ms/step - loss: 36.6988 - root_mean_squared_error: 6.0580 - val_loss: 0.8164

#### Wide and Deep model
*Using functional API - branching*

In [6]:
norm_layer = tf.keras.layers.Normalization()
hlayer_1 = tf.keras.layers.Dense(30, activation="relu")
hlayer_3 = tf.keras.layers.Dense(30, activation="relu")
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(1)

In [7]:
input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
normalized = norm_layer(input_)
hidden1 = hlayer_1(normalized)
hidden2 = hlayer_3(hidden1)
concat = concat_layer([hidden2, normalized])
output = output_layer(concat)

2024-01-28 09:29:15.310010: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-01-28 09:29:15.310034: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-01-28 09:29:15.310040: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-01-28 09:29:15.310104: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-28 09:29:15.310317: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
model_wide = tf.keras.Model(inputs=[input_], outputs=[output])

In [11]:
model_wide.compile(loss="mse", optimizer="adam", metrics=["RootMeanSquaredError"])

In [12]:
norm_layer.adapt(X_train)
model_wide.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30
516/516 [==============================] - 4s 7ms/step - loss: 1.6370 - root_mean_squared_error: 1.2794 - val_loss: 0.5803 - val_root_mean_squared_error: 0.7618
Epoch 2/30
516/516 [==============================] - 3s 7ms/step - loss: 0.6790 - root_mean_squared_error: 0.8240 - val_loss: 0.6242 - val_root_mean_squared_error: 0.7901
Epoch 3/30
516/516 [==============================] - 3s 6ms/step - loss: 0.6087 - root_mean_squared_error: 0.7802 - val_loss: 0.5632 - val_root_mean_squared_error: 0.7505
Epoch 4/30
516/516 [==============================] - 3s 6ms/step - loss: 0.8583 - root_mean_squared_error: 0.9265 - val_loss: 0.5607 - val_root_mean_squared_error: 0.7488
Epoch 5/30
516/516 [==============================] - 3s 6ms/step - loss: 0.6026 - root_mean_squared_error: 0.7763 - val_loss: 0.6233 - val_root_mean_squared_error: 0.7895
Epoch 6/30
516/516 [==============================] - 3s 6ms/step - loss: 1.3424 - root_mean_squared_error: 1.1586 - val_loss: 0.5624 - val_

### Deep and Wide Model
*Using Keras Functional API - multi input approach*

In [6]:
input_wide = tf.keras.layers.Input(shape=[5], name="input_wide")
input_deep = tf.keras.layers.Input(shape=[6], name="input_deep")

norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()

norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

h1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
h2 = tf.keras.layers.Dense(30, activation="relu")(h1)
concat_ = tf.keras.layers.concatenate([norm_wide, h2])
output_ = tf.keras.layers.Dense(1)(concat_)

model_split_features = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output_])

2024-01-28 10:40:02.451548: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-01-28 10:40:02.451578: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-01-28 10:40:02.451583: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-01-28 10:40:02.451660: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-28 10:40:02.451850: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]

In [14]:
model_split_features.compile(loss="mse", optimizer="adam", metrics=["RootMeanSquaredError"])

In [15]:
hist_fsplit = model_split_features.fit((X_train_wide, X_train_deep), y_train, epochs=30, validation_data=((X_test_wide, X_test_deep), y_test))
# hist_fsplit = model_split_features.fit({"input_wide": X_train_wide, "input_deep":X_train_deep}, y_train, epochs=30, validation_data=({"input_wide": X_test_wide, "input_deep": X_test_deep}, y_test))

Epoch 1/30
516/516 [==============================] - 4s 7ms/step - loss: 831.7415 - root_mean_squared_error: 28.8399 - val_loss: 5.7120 - val_root_mean_squared_error: 2.3900
Epoch 2/30
516/516 [==============================] - 3s 7ms/step - loss: 8.4137 - root_mean_squared_error: 2.9006 - val_loss: 4.2034 - val_root_mean_squared_error: 2.0502
Epoch 3/30
516/516 [==============================] - 3s 7ms/step - loss: 6.9722 - root_mean_squared_error: 2.6405 - val_loss: 1.9510 - val_root_mean_squared_error: 1.3968
Epoch 4/30
516/516 [==============================] - 3s 7ms/step - loss: 3.6953 - root_mean_squared_error: 1.9223 - val_loss: 2.3898 - val_root_mean_squared_error: 1.5459
Epoch 5/30
516/516 [==============================] - 3s 6ms/step - loss: 12.9075 - root_mean_squared_error: 3.5927 - val_loss: 1.6287 - val_root_mean_squared_error: 1.2762
Epoch 6/30
516/516 [==============================] - 3s 6ms/step - loss: 6.4923 - root_mean_squared_error: 2.5480 - val_loss: 344.0101 